In [4]:
# === CONFIGURACIÓN INICIAL ===
import numpy as np
import pandas as pd
import sys
import os
import math

# Imports estándar
import matplotlib.pyplot as plt
import seaborn as sns


# Obtener la ruta raíz del proyecto de forma más robusta
# Si estás en notebooks/, sube un nivel para llegar a la raíz
notebook_dir = os.getcwd()  # Directorio actual del notebook
ruta_raiz = os.path.dirname(notebook_dir)  # Sube un nivel (a la raíz del proyecto)

# Si ya estás en la raíz, usa directamente:
# ruta_raiz = os.getcwd()

print(f"Ruta del notebook: {notebook_dir}")
print(f"Ruta raíz: {ruta_raiz}")

# Agregar la ruta raíz al path de Python
if ruta_raiz not in sys.path:
    sys.path.insert(0, ruta_raiz)



# Importar funciones del proyecto
from src.data.preprocessing import (
    verificar_columna_numerica,
    verificar_multiples_columnas
)

# Configuración de visualización
%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

print("✓ Configuración completada")

Ruta del notebook: /home/alexander/Escritorio/desercion-universitaria/notebooks
Ruta raíz: /home/alexander/Escritorio/desercion-universitaria
✓ Configuración completada


En este notebook se hara todo el proceso para crear una base de datos 
general con todas las variable explicativas que nuego pasaran  
a implementacion de modelos de aprendizaje 

In [3]:
#Datos limpios de estudiates con historial de maricula, repeticiones y promedios por periodo academico matriculado.

data_est_clean = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/raw/Datos_limpios_estudiantes.csv') 
data_est_clean.head()


,APELLIDOS_NOMBRES,IDENTIFICACION,CODIGO,PERIODO,NUM_PERIODOS,MATERIAS,CREDITOS_MATRICULADOS,APROBADAS,CREDITOS_APROBADOS,PERDIDAS,R1,R2,R3,R3_BALANCEO,PROMEDIO
0,ACHINTE MARTINEZ JULIETH ALEXANDRA,1002958067,106217020643,2017.2,NaN,4,14,2,6,2,0,0,0,0.0,2.975
1,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2008.2,NaN,4,14,3,10,1,0,0,0,0.0,3.300
2,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.1,NaN,5,15,5,15,0,1,0,0,0.0,3.425
3,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.2,NaN,4,16,2,8,2,0,0,0,0.0,2.875
4,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2010.1,NaN,4,16,3,12,1,2,0,0,0.0,3.200


1. verificacion de formatos y valores de los datos por columnas.

In [4]:


def verificar_datos_numericos(df, desde_columna = 1):
    """
    Verifica si las columnas de un DataFrame son numéricas,
    comenzando desde la columna indicada (por defecto, la segunda).
    Si encuentra valores no numéricos o faltantes, muestra su ubicación exacta.
    """
    print("=== Verificación de datos no numéricos y faltantes ===\n")

    errores = []
    faltantes_info = []

    for col in df.columns[desde_columna:]:
        # Intentar convertir a numérico (sin modificar el original)
        serie_numerica = pd.to_numeric(df[col], errors='coerce')

        # --- Detectar valores no numéricos ---
        no_numericos = df[col][serie_numerica.isna() & df[col].notna()]
        if len(no_numericos) > 0:
            print(f"⚠️ Columna '{col}' contiene valores no numéricos:")
            for idx, val in no_numericos.items():
                print(f"   → Fila {idx}, Valor: '{val}'")
                errores.append((idx, col, val))
            print()

        # --- Detectar valores faltantes ---
        faltantes = df[col][df[col].isna()]
        if len(faltantes) > 0:
            print(f"🔹 Columna '{col}' tiene {len(faltantes)} valores faltantes:")
            for idx in faltantes.index:
                print(f"   → Fila {idx} (vacío o NaN)")
                faltantes_info.append((idx, col))
            print()

    # --- Resumen ---
    if not errores:
        print("✅ No se encontraron valores no numéricos.")
    else:
        print(f"Se encontraron {len(errores)} valores no numéricos en total.")

    if not faltantes_info:
        print("✅ No se encontraron valores faltantes.")
    else:
        print(f"Se encontraron {len(faltantes_info)} valores faltantes en total.")

    return errores, faltantes_info


In [5]:
verificar_datos_numericos(data_est_clean, desde_columna = 1)

=== Verificación de datos no numéricos y faltantes ===

🔹 Columna 'NUM_PERIODOS' tiene 8718 valores faltantes:
   → Fila 0 (vacío o NaN)
   → Fila 1 (vacío o NaN)
   → Fila 2 (vacío o NaN)
   → Fila 3 (vacío o NaN)
   → Fila 4 (vacío o NaN)
   → Fila 5 (vacío o NaN)
   → Fila 6 (vacío o NaN)
   → Fila 7 (vacío o NaN)
   → Fila 8 (vacío o NaN)
   → Fila 9 (vacío o NaN)
   → Fila 10 (vacío o NaN)
   → Fila 11 (vacío o NaN)
   → Fila 12 (vacío o NaN)
   → Fila 13 (vacío o NaN)
   → Fila 14 (vacío o NaN)
   → Fila 15 (vacío o NaN)
   → Fila 16 (vacío o NaN)
   → Fila 17 (vacío o NaN)
   → Fila 18 (vacío o NaN)
   → Fila 19 (vacío o NaN)
   → Fila 20 (vacío o NaN)
   → Fila 21 (vacío o NaN)
   → Fila 22 (vacío o NaN)
   → Fila 23 (vacío o NaN)
   → Fila 24 (vacío o NaN)
   → Fila 25 (vacío o NaN)
   → Fila 26 (vacío o NaN)
   → Fila 27 (vacío o NaN)
   → Fila 28 (vacío o NaN)
   → Fila 29 (vacío o NaN)
   → Fila 30 (vacío o NaN)
   → Fila 31 (vacío o NaN)
   → Fila 32 (vacío o NaN)
   → Fil

([],
 [(0, 'NUM_PERIODOS'),
  (1, 'NUM_PERIODOS'),
  (2, 'NUM_PERIODOS'),
  (3, 'NUM_PERIODOS'),
  (4, 'NUM_PERIODOS'),
  (5, 'NUM_PERIODOS'),
  (6, 'NUM_PERIODOS'),
  (7, 'NUM_PERIODOS'),
  (8, 'NUM_PERIODOS'),
  (9, 'NUM_PERIODOS'),
  (10, 'NUM_PERIODOS'),
  (11, 'NUM_PERIODOS'),
  (12, 'NUM_PERIODOS'),
  (13, 'NUM_PERIODOS'),
  (14, 'NUM_PERIODOS'),
  (15, 'NUM_PERIODOS'),
  (16, 'NUM_PERIODOS'),
  (17, 'NUM_PERIODOS'),
  (18, 'NUM_PERIODOS'),
  (19, 'NUM_PERIODOS'),
  (20, 'NUM_PERIODOS'),
  (21, 'NUM_PERIODOS'),
  (22, 'NUM_PERIODOS'),
  (23, 'NUM_PERIODOS'),
  (24, 'NUM_PERIODOS'),
  (25, 'NUM_PERIODOS'),
  (26, 'NUM_PERIODOS'),
  (27, 'NUM_PERIODOS'),
  (28, 'NUM_PERIODOS'),
  (29, 'NUM_PERIODOS'),
  (30, 'NUM_PERIODOS'),
  (31, 'NUM_PERIODOS'),
  (32, 'NUM_PERIODOS'),
  (33, 'NUM_PERIODOS'),
  (34, 'NUM_PERIODOS'),
  (35, 'NUM_PERIODOS'),
  (36, 'NUM_PERIODOS'),
  (37, 'NUM_PERIODOS'),
  (38, 'NUM_PERIODOS'),
  (39, 'NUM_PERIODOS'),
  (40, 'NUM_PERIODOS'),
  (41, 'NUM_PERIODOS'

**Regularizacion de estudiantes que ahn hecho transito entre programas**

In [6]:
def regularizar_num_periodos(df, col_codigo='CODIGO', col_creditos='CREDITOS_MATRICULADOS', 
                              col_num_periodos='NUM_PERIODOS', col_nombres='APELLIDOS_NOMBRES',
                              umbral=14, divisor=16.6, guardar=True, 
                              ruta_salida='/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_periodos_regularizados.csv'):
    """
    Regulariza el número de períodos académicos de cada estudiante.
    
    Para estudiantes que en su primer período tienen más de 'umbral' créditos matriculados,
    calcula el número inicial de períodos como ceil(creditos_primer_periodo / divisor).
    Para los demás, comienza en 1.
    
    El DataFrame resultante se ordena alfabéticamente por apellidos y nombres.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos de estudiantes
        col_codigo (str): Nombre de la columna con el código del estudiante
        col_creditos (str): Nombre de la columna con créditos matriculados
        col_num_periodos (str): Nombre de la columna donde guardar el número de períodos
        col_nombres (str): Nombre de la columna con apellidos y nombres para ordenar
        umbral (int): Umbral de créditos para aplicar la regularización (default: 14)
        divisor (float): Divisor para calcular períodos equivalentes (default: 16.6)
        guardar (bool): Si True, guarda el resultado en un archivo CSV
        ruta_salida (str): Ruta donde guardar el archivo CSV
    
    Returns:
        pd.DataFrame: DataFrame con la columna NUM_PERIODOS actualizada y ordenado alfabéticamente
    
    Examples:
        >>> df = regularizar_num_periodos(df, guardar=True)
        >>> # Estudiante con 65 créditos en primer período: n = ceil(65/16.6) = 4
        >>> # Estudiante con 14 créditos en primer período: n = 1
    """
    
    # Crear copia para no modificar el original
    df_result = df.copy()
    
    # Asegurar que la columna existe
    if col_num_periodos not in df_result.columns:
        df_result[col_num_periodos] = np.nan
    
    # Ordenar por código de estudiante y período para procesar en orden cronológico
    df_result = df_result.sort_values([col_codigo, 'PERIODO']).reset_index(drop=True)
    
    # Procesar cada estudiante
    estudiantes_unicos = df_result[col_codigo].unique()
    
    print(f"Regularizando períodos para {len(estudiantes_unicos)} estudiantes...")
    
    estudiantes_regularizados = 0
    estudiantes_normales = 0
    
    for codigo in estudiantes_unicos:
        # Filtrar registros del estudiante
        mask_estudiante = df_result[col_codigo] == codigo
        indices_estudiante = df_result[mask_estudiante].index
        
        # Obtener créditos del primer período
        primer_indice = indices_estudiante[0]
        creditos_primer_periodo = df_result.loc[primer_indice, col_creditos]
        
        # Determinar el número de período inicial
        if creditos_primer_periodo > umbral:
            # Regularización: calcular períodos equivalentes
            periodos_equivalentes = creditos_primer_periodo / divisor
            num_periodo_inicial = math.ceil(periodos_equivalentes)
            estudiantes_regularizados += 1
        else:
            # Sin regularización: comenzar en 1
            num_periodo_inicial = 1
            estudiantes_normales += 1
        
        # Asignar números de período consecutivos
        for i, idx in enumerate(indices_estudiante):
            df_result.loc[idx, col_num_periodos] = num_periodo_inicial + i
    
    # Convertir a entero
    df_result[col_num_periodos] = df_result[col_num_periodos].astype(int)
    
    # Ordenar alfabéticamente por apellidos y nombres, manteniendo el orden por período dentro de cada estudiante
    print(f"\nOrdenando alfabéticamente por {col_nombres}...")
    df_result = df_result.sort_values([col_nombres, 'PERIODO']).reset_index(drop=True)
    
    # Reporte
    print(f"\n✓ Regularización completada:")
    print(f"  - Estudiantes con regularización (>{umbral} créditos): {estudiantes_regularizados}")
    print(f"  - Estudiantes sin regularización (≤{umbral} créditos): {estudiantes_normales}")
    print(f"  - Total procesado: {len(estudiantes_unicos)}")
    print(f"  - Datos ordenados alfabéticamente por {col_nombres}")
    
    # Guardar si se solicita
    if guardar:
        import os
        # Crear el directorio si no existe
        directorio = os.path.dirname(ruta_salida)
        if directorio and not os.path.exists(directorio):
            os.makedirs(directorio)
            print(f"\n📁 Directorio creado: {directorio}")
        
        df_result.to_csv(ruta_salida, index=False)
        print(f"\n💾 Archivo guardado exitosamente en: {ruta_salida}")
        print(f"   - Total de registros: {len(df_result)}")
        print(f"   - Tamaño del archivo: {os.path.getsize(ruta_salida) / 1024:.2f} KB")
    
    return df_result


def verificar_regularizacion(df, col_codigo='CODIGO', col_creditos='CREDITOS_MATRICULADOS', 
                              col_num_periodos='NUM_PERIODOS', mostrar_ejemplos=5):
    """
    Verifica y muestra ejemplos de la regularización aplicada.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos regularizados
        col_codigo (str): Nombre de la columna con el código del estudiante
        col_creditos (str): Nombre de la columna con créditos matriculados
        col_num_periodos (str): Nombre de la columna con número de períodos
        mostrar_ejemplos (int): Número de ejemplos a mostrar de cada tipo
    
    Returns:
        dict: Diccionario con estadísticas de la verificación
    """
    
    print("=" * 80)
    print("VERIFICACIÓN DE REGULARIZACIÓN DE PERÍODOS")
    print("=" * 80)
    
    # Obtener primer registro de cada estudiante
    primeros_registros = df.groupby(col_codigo).first().reset_index()
    
    # Clasificar estudiantes
    regularizados = primeros_registros[primeros_registros[col_creditos] > 14]
    normales = primeros_registros[primeros_registros[col_creditos] <= 14]
    
    print(f"\n📊 Resumen:")
    print(f"  - Total estudiantes: {len(primeros_registros)}")
    print(f"  - Con regularización: {len(regularizados)} ({len(regularizados)/len(primeros_registros)*100:.1f}%)")
    print(f"  - Sin regularización: {len(normales)} ({len(normales)/len(primeros_registros)*100:.1f}%)")
    
    # Mostrar ejemplos de estudiantes regularizados
    if len(regularizados) > 0:
        print(f"\n{'='*80}")
        print(f"EJEMPLOS DE ESTUDIANTES REGULARIZADOS (primeros {min(mostrar_ejemplos, len(regularizados))}):")
        print(f"{'='*80}")
        print(f"{'Código':<15} {'Créditos 1er periodo':<25} {'Período inicial':<20} {'Cálculo'}")
        print(f"{'-'*80}")
        
        for _, row in regularizados.head(mostrar_ejemplos).iterrows():
            creditos = row[col_creditos]
            periodo_inicial = row[col_num_periodos]
            calculo = creditos / 16.6
            print(f"{row[col_codigo]:<15} {creditos:<25} {periodo_inicial:<20} ceil({calculo:.2f})")
    
    # Mostrar ejemplos de estudiantes normales
    if len(normales) > 0:
        print(f"\n{'='*80}")
        print(f"EJEMPLOS DE ESTUDIANTES SIN REGULARIZACIÓN (primeros {min(mostrar_ejemplos, len(normales))}):")
        print(f"{'='*80}")
        print(f"{'Código':<15} {'Créditos 1er periodo':<25} {'Período inicial'}")
        print(f"{'-'*80}")
        
        for _, row in normales.head(mostrar_ejemplos).iterrows():
            print(f"{row[col_codigo]:<15} {row[col_creditos]:<25} {row[col_num_periodos]}")
    
    print(f"\n{'='*80}\n")
    
    # Retornar estadísticas
    return {
        'total_estudiantes': len(primeros_registros),
        'regularizados': len(regularizados),
        'normales': len(normales),
        'porcentaje_regularizados': len(regularizados)/len(primeros_registros)*100 if len(primeros_registros) > 0 else 0
    }

In [ ]:
df  = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/raw/Datos_limpios_estudiantes.csv')


In [8]:
regularizar_num_periodos(df, col_codigo='CODIGO', col_creditos='CREDITOS_MATRICULADOS', 
                              col_num_periodos='NUM_PERIODOS', col_nombres='APELLIDOS_NOMBRES',
                              umbral=14, divisor=16.6, guardar=False, 
                              ruta_salida='/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_periodos_regularizados.csv')

Regularizando períodos para 1485 estudiantes...

Ordenando alfabéticamente por APELLIDOS_NOMBRES...

✓ Regularización completada:
  - Estudiantes con regularización (>14 créditos): 74
  - Estudiantes sin regularización (≤14 créditos): 1411
  - Total procesado: 1485
  - Datos ordenados alfabéticamente por APELLIDOS_NOMBRES


,APELLIDOS_NOMBRES,IDENTIFICACION,CODIGO,PERIODO,NUM_PERIODOS,MATERIAS,CREDITOS_MATRICULADOS,APROBADAS,CREDITOS_APROBADOS,PERDIDAS,R1,R2,R3,R3_BALANCEO,PROMEDIO
0,ACHINTE MARTINEZ JULIETH ALEXANDRA,1002958067,106217020643,2017.2,1,4,14,2,6,2,0,0,0,0.0,2.975
1,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2008.2,1,4,14,3,10,1,0,0,0,0.0,3.300
2,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.1,2,5,15,5,15,0,1,0,0,0.0,3.425
3,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.2,3,4,16,2,8,2,0,0,0,0.0,2.875
4,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2010.1,4,4,16,3,12,1,2,0,0,0.0,3.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8713,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.1,4,3,12,0,0,3,3,0,0,0.0,0.700
8714,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.2,5,2,8,0,0,2,0,2,0,0.0,0.000
8715,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.1,1,5,14,5,14,0,0,0,0,0.0,4.000
8716,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.2,2,5,15,4,11,1,0,0,0,0.0,3.700


**Creación de la columna para acumular los créditos aprobados por estudiante y periodo académico.**

In [22]:
def calcular_creditos_acumulados(df, col_codigo='CODIGO', col_creditos_aprobados='CREDITOS_APROBADOS',
                                  col_nombres='APELLIDOS_NOMBRES', col_periodo='PERIODO',
                                  col_acumulado='CREDITOS_APROBADOS_ACUM',
                                  guardar=True, ruta_salida='/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_creditos_acumulados.csv'):
    """
    Calcula el acumulado de créditos aprobados para cada estudiante período a período.
    
    
    Para cada estudiante, suma los créditos aprobados de forma acumulativa a través
    de sus períodos académicos. El DataFrame mantiene el orden alfabético por apellidos y nombres.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos de estudiantes
        col_codigo (str): Nombre de la columna con el código del estudiante
        col_creditos_aprobados (str): Nombre de la columna con créditos aprobados por período
        col_nombres (str): Nombre de la columna con apellidos y nombres
        col_periodo (str): Nombre de la columna con el período académico
        col_acumulado (str): Nombre de la nueva columna con el acumulado
        guardar (bool): Si True, guarda el resultado en un archivo CSV
        ruta_salida (str): Ruta donde guardar el archivo CSV
    
    Returns:
        pd.DataFrame: DataFrame con la nueva columna de créditos acumulados
    
    Examples:
        >>> df = calcular_creditos_acumulados(df, guardar=True)
        >>> # Estudiante en periodo 1: acum = 15
        >>> # Estudiante en periodo 2: acum = 15 + 18 = 33
        >>> # Estudiante en periodo 3: acum = 33 + 16 = 49
    """
    
    # Crear copia para no modificar el original
    df_result = df.copy()
    
    print("=" * 80)
    print("CALCULANDO CRÉDITOS APROBADOS ACUMULADOS")
    print("=" * 80)
    
    # Verificar que las columnas necesarias existen
    columnas_requeridas = [col_codigo, col_creditos_aprobados, col_nombres, col_periodo]
    columnas_faltantes = [col for col in columnas_requeridas if col not in df_result.columns]
    
    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes en el DataFrame: {columnas_faltantes}")
    
    # Asegurar que el DataFrame está ordenado correctamente
    # Primero por nombres alfabéticamente, luego por período dentro de cada estudiante
    df_result = df_result.sort_values([col_nombres, col_periodo]).reset_index(drop=True)
    
    # Crear la columna de acumulado
    df_result[col_acumulado] = 0
    
    # Obtener estudiantes únicos manteniendo el orden alfabético
    estudiantes_unicos = df_result[col_codigo].unique()
    
    print(f"\nProcesando {len(estudiantes_unicos)} estudiantes...")
    
    # Calcular acumulado para cada estudiante
    for codigo in estudiantes_unicos:
        # Obtener índices del estudiante (ya están ordenados por período)
        mask_estudiante = df_result[col_codigo] == codigo
        indices_estudiante = df_result[mask_estudiante].index
        
        # Calcular acumulado
        acumulado = 0
        for idx in indices_estudiante:
            creditos_periodo = df_result.loc[idx, col_creditos_aprobados]
            acumulado += creditos_periodo
            df_result.loc[idx, col_acumulado] = acumulado
    
    # Convertir a entero (asumiendo que los créditos son números enteros)
    df_result[col_acumulado] = df_result[col_acumulado].astype(int)
    
    # Estadísticas
    total_registros = len(df_result)
    creditos_max = df_result[col_acumulado].max()
    creditos_min = df_result[col_acumulado].min()
    creditos_promedio = df_result[col_acumulado].mean()
    
    print(f"\n✓ Cálculo completado:")
    print(f"  - Total de registros procesados: {total_registros}")
    print(f"  - Total de estudiantes: {len(estudiantes_unicos)}")
    print(f"  - Créditos acumulados máximos: {creditos_max}")
    print(f"  - Créditos acumulados mínimos: {creditos_min}")
    print(f"  - Promedio de créditos acumulados: {creditos_promedio:.2f}")
    print(f"  - Datos mantienen orden alfabético por {col_nombres}")
    
    # Guardar si se solicita
    if guardar:
        import os
        # Crear el directorio si no existe
        directorio = os.path.dirname(ruta_salida)
        if directorio and not os.path.exists(directorio):
            os.makedirs(directorio)
            print(f"\n📁 Directorio creado: {directorio}")
        
        df_result.to_csv(ruta_salida, index=False)
        print(f"\n💾 Archivo guardado exitosamente en: {ruta_salida}")
        print(f"   - Total de registros: {len(df_result)}")
        print(f"   - Tamaño del archivo: {os.path.getsize(ruta_salida) / 1024:.2f} KB")
    
    print("\n" + "=" * 80 + "\n")
    
    return df_result

In [28]:
df1  = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_periodos_regularizados.csv')


calcular_creditos_acumulados(df1, col_codigo='CODIGO', col_creditos_aprobados='CREDITOS_APROBADOS',
                                  col_nombres='APELLIDOS_NOMBRES', col_periodo='PERIODO',
                                  col_acumulado='CREDITOS_APROBADOS_ACUM',
                                  guardar=True, ruta_salida='/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_creditos_acumulados.csv')

CALCULANDO CRÉDITOS APROBADOS ACUMULADOS

Procesando 1485 estudiantes...

✓ Cálculo completado:
  - Total de registros procesados: 8718
  - Total de estudiantes: 1485
  - Créditos acumulados máximos: 213
  - Créditos acumulados mínimos: 0
  - Promedio de créditos acumulados: 61.81
  - Datos mantienen orden alfabético por APELLIDOS_NOMBRES

💾 Archivo guardado exitosamente en: /home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_creditos_acumulados.csv
   - Total de registros: 8718
   - Tamaño del archivo: 763.71 KB




,APELLIDOS_NOMBRES,IDENTIFICACION,CODIGO,PERIODO,NUM_PERIODOS,MATERIAS,CREDITOS_MATRICULADOS,APROBADAS,CREDITOS_APROBADOS,PERDIDAS,R1,R2,R3,R3_BALANCEO,PROMEDIO,CREDITOS_APROBADOS_ACUM
0,ACHINTE MARTINEZ JULIETH ALEXANDRA,1002958067,106217020643,2017.2,1,4,14,2,6,2,0,0,0,0.0,2.975,6
1,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2008.2,1,4,14,3,10,1,0,0,0,0.0,3.300,10
2,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.1,2,5,15,5,15,0,1,0,0,0.0,3.425,25
3,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.2,3,4,16,2,8,2,0,0,0,0.0,2.875,33
4,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2010.1,4,4,16,3,12,1,2,0,0,0.0,3.200,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8713,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.1,4,3,12,0,0,3,3,0,0,0.0,0.700,13
8714,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.2,5,2,8,0,0,2,0,2,0,0.0,0.000,13
8715,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.1,1,5,14,5,14,0,0,0,0,0.0,4.000,14
8716,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.2,2,5,15,4,11,1,0,0,0,0.0,3.700,25


**Script para la creación del indicador de EFICIENCIA ACADÉMICA**

$$CAP = \frac{\text{ Num Créditos aprobados (acumulados)}}{\text{Num Periodos matriculados}}$$
$$CRP = \frac{\text{Num créditos del plan de estudios}}{\text{Número de periodos del plan de estudios}}$$

$$EFICIENCIA = \frac{CAP}{CRP}$$

El indicador de eficiencia se calcula en cada periodo académico matriculado por un estudiante.

In [ ]:
def calcular_eficiencia_academica(df, guardar=True):
    
    """
    Calcula la eficiencia académica de los estudiantes en función de los créditos aprobados.

    Args:
        df (pd.DataFrame): DataFrame con los datos de estudiantes

    Returns:
        pd.DataFrame: DataFrame con la eficiencia académica de los estudiantes
    """

    ruta_salida='/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_eficiencia_academica.csv'

    # Crear copia para no modificar el original
    df_result = df.copy()

    # Verificar que las columnas necesarias existen
    columnas_requeridas = ['CREDITOS_APROBADOS', 'NUM_PERIODOS']
    columnas_faltantes = [col for col in columnas_requeridas if col not in df_result.columns]

    if columnas_faltantes:
        raise ValueError(f"Columnas faltantes en el DataFrame: {columnas_faltantes}")

    # Asegurar que el DataFrame está ordenado correctamente
    df_result = df_result.sort_values(['APELLIDOS_NOMBRES', 'PERIODO']).reset_index(drop=True)

    CAP = df['CREDITOS_APROBADOS_ACUM'] / df['NUM_PERIODOS']
    CRP = 16.6  

    # Calcular la eficiencia académica
    df_result['EFICIENCIA_ACADEMICA'] = CAP / CRP

    if guardar:
        import os
        # Crear el directorio si no existe
        directorio = os.path.dirname(ruta_salida)
        if directorio and not os.path.exists(directorio):
            os.makedirs(directorio)
            print(f"\n📁 Directorio creado: {directorio}")

        df_result.to_csv(ruta_salida, index=False)
        print(f"\n💾 Archivo guardado exitosamente en: {ruta_salida}")
        print(f"   - Total de registros: {len(df_result)}")
        print(f"   - Tamaño del archivo: {os.path.getsize(ruta_salida) / 1024:.2f} KB")

    print(f"\n✓ Cálculo completado:")
    print(f"  - Total de registros procesados: {len(df_result)}")
    print(f"  - Total de estudiantes: {df_result['CODIGO'].nunique()}")
    print(f"  - Eficiencia académica promedio: {df_result['EFICIENCIA_ACADEMICA'].mean():.2f}")

    print("\n" + "=" * 80 + "\n")

    return df_result
    

In [30]:
df2 = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_creditos_acumulados.csv')


calcular_eficiencia_academica(df2)

CALCULANDO EFICIENCIA ACADÉMICA

💾 Archivo guardado exitosamente en: /home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_eficiencia_academica.csv
   - Total de registros: 8718
   - Tamaño del archivo: 923.97 KB

✓ Cálculo completado:
  - Total de registros procesados: 8718
  - Total de estudiantes: 1485
  - Eficiencia académica promedio: 0.60




,APELLIDOS_NOMBRES,IDENTIFICACION,CODIGO,PERIODO,NUM_PERIODOS,MATERIAS,CREDITOS_MATRICULADOS,APROBADAS,CREDITOS_APROBADOS,PERDIDAS,R1,R2,R3,R3_BALANCEO,PROMEDIO,CREDITOS_APROBADOS_ACUM,EFICIENCIA_ACADEMICA
0,ACHINTE MARTINEZ JULIETH ALEXANDRA,1002958067,106217020643,2017.2,1,4,14,2,6,2,0,0,0,0.0,2.975,6,0.361446
1,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2008.2,1,4,14,3,10,1,0,0,0,0.0,3.300,10,0.602410
2,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.1,2,5,15,5,15,0,1,0,0,0.0,3.425,25,0.753012
3,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.2,3,4,16,2,8,2,0,0,0,0.0,2.875,33,0.662651
4,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2010.1,4,4,16,3,12,1,2,0,0,0.0,3.200,45,0.677711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8713,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.1,4,3,12,0,0,3,3,0,0,0.0,0.700,13,0.195783
8714,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.2,5,2,8,0,0,2,0,2,0,0.0,0.000,13,0.156627
8715,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.1,1,5,14,5,14,0,0,0,0,0.0,4.000,14,0.843373
8716,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.2,2,5,15,4,11,1,0,0,0,0.0,3.700,25,0.753012


**Calculo del promedio de carrera**

In [7]:


def calcular_promedio_carrera(df,guardar = True):
    """
    Calcula el promedio acumulado de carrera por estudiante ('CODIGO')
    sin alterar el orden alfabético de 'APELLIDOS_NOMBRES'.
    """
    # Verificar columnas necesarias
    columnas_necesarias = {'CODIGO', 'PROMEDIO'}
    if not columnas_necesarias.issubset(df.columns):
        raise ValueError(f"Faltan columnas necesarias: {columnas_necesarias - set(df.columns)}")

    ruta_salida = '/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_prom_carrera.csv'
    # Calcular promedio acumulado por estudiante
    df['PROMEDIO_CARRERA'] = (
        df.groupby('CODIGO')['PROMEDIO']
          .expanding()
          .mean()
          .reset_index(level=0, drop=True)
    )

    # Mantener el orden original (alfabético por nombres)
    df = df.sort_values('APELLIDOS_NOMBRES', kind='stable').reset_index(drop=True)

    if guardar:
        import os
        # Crear el directorio si no existe
        directorio = os.path.dirname(ruta_salida)
        if directorio and not os.path.exists(directorio):
            os.makedirs(directorio)
            print(f"\n📁 Directorio creado: {directorio}")

        df.to_csv(ruta_salida, index=False)
        print(f"\n💾 Archivo guardado exitosamente en: {ruta_salida}")
        print(f"   - Total de registros: {len(df)}")
        print(f"   - Tamaño del archivo: {os.path.getsize(ruta_salida) / 1024:.2f} KB")
    
    return df


In [8]:
df3 = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_eficiencia_academica.csv')

calcular_promedio_carrera(df3, guardar= True)


💾 Archivo guardado exitosamente en: /home/alexander/Escritorio/desercion-universitaria/data/processed/estudiantes_prom_carrera.csv
   - Total de registros: 8718
   - Tamaño del archivo: 1047.81 KB


,APELLIDOS_NOMBRES,IDENTIFICACION,CODIGO,PERIODO,NUM_PERIODOS,MATERIAS,CREDITOS_MATRICULADOS,APROBADAS,CREDITOS_APROBADOS,PERDIDAS,R1,R2,R3,R3_BALANCEO,PROMEDIO,CREDITOS_APROBADOS_ACUM,EFICIENCIA_ACADEMICA,PROMEDIO_CARRERA
0,ACHINTE MARTINEZ JULIETH ALEXANDRA,1002958067,106217020643,2017.2,1,4,14,2,6,2,0,0,0,0.0,2.975,6,0.361446,2.975000
1,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2008.2,1,4,14,3,10,1,0,0,0,0.0,3.300,10,0.602410,3.300000
2,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.1,2,5,15,5,15,0,1,0,0,0.0,3.425,25,0.753012,3.362500
3,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2009.2,3,4,16,2,8,2,0,0,0,0.0,2.875,33,0.662651,3.200000
4,ACHIPIZ QUINA YOHN EDWAR,1061709826,62082037,2010.1,4,4,16,3,12,1,2,0,0,0.0,3.200,45,0.677711,3.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8713,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.1,4,3,12,0,0,3,3,0,0,0.0,0.700,13,0.195783,1.535500
8714,ZUÑIGA ZAMBRANO ANDRES MAURICIO,1064676332,62052012,2007.2,5,2,8,0,0,2,0,2,0,0.0,0.000,13,0.156627,1.228400
8715,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.1,1,5,14,5,14,0,0,0,0,0.0,4.000,14,0.843373,4.000000
8716,ÁLVAREZ MAZABEL JUAN ANDRÉS,1084330130,106221010943,2021.2,2,5,15,4,11,1,0,0,0,0.0,3.700,25,0.753012,3.850000


In [ ]:


def obtener_ultima_entrada_por_estudiante(df, columnas_interes):
    """
    Retorna un DataFrame con la última entrada (último período) de cada estudiante.
    
    Parámetros
    ----------
    df : pandas.DataFrame
        Base de datos completa con todas las entradas por período.
    columnas_interes : list
        Lista de columnas que se desean conservar (además de CODIGO).
        Ejemplo: ['APELLIDOS_NOMBRES', 'PERIODO', 'PROMEDIO_CARRERA', 'CREDITOS_APROBADOS']
    
    Retorna
    -------
    pandas.DataFrame con una fila por estudiante.
    """
    # Verificar columnas
    columnas_necesarias = ['CODIGO'] + columnas_interes
    for col in columnas_necesarias:
        if col not in df.columns:
            raise ValueError(f"Falta la columna '{col}' en el DataFrame.")
    
    # Ordenar por CODIGO y PERIODO (para que 'last' funcione bien)
    # Asegúrate de que la columna de período esté en formato ordenable (string o número)
    if 'PERIODO' in df.columns:
        df = df.sort_values(['CODIGO', 'PERIODO'])
    else:
        df = df.sort_values('CODIGO')
    
    # Tomar la última fila de cada estudiante
    ultimos = df.groupby('CODIGO').tail(1)[columnas_necesarias]
    
    # Mantener el orden alfabético por nombres
    if 'APELLIDOS_NOMBRES' in ultimos.columns:
        ultimos = ultimos.sort_values('APELLIDOS_NOMBRES', kind='stable').reset_index(drop=True)
    
    return ultimos



#ejeplo de uso 

'''
df = pd.read_csv("estudiantes_eficiencia_academica.csv")

# Definir las columnas que te interesan conservar
columnas = ['APELLIDOS_NOMBRES', 'PERIODO', 'PROMEDIO_CARRERA', 'CREDITOS_APROBADOS']

# Obtener una tabla con el último registro de cada estudiante
resumen_final = obtener_ultima_entrada_por_estudiante(df, columnas)

# Ver resultado
print(resumen_final.head())

# Guardar si lo deseas
resumen_final.to_csv("resumen_final_estudiantes.csv", index=False)
'''

In [14]:
df_pruebas = pd.read_csv('/home/alexander/Escritorio/desercion-universitaria/data/features/df_pruebas.csv')

In [15]:
import pandas as pd

def crear_base_general(df):
    """
    Crea una base de datos condensada (una fila por estudiante) a partir de la base original.
    """

    # --- Agrupar por CODIGO ---
    resumen = df.groupby('CODIGO').agg({
        'APELLIDOS_NOMBRES': 'first',               # nombres y apellidos
        'IDENTIFICACION': 'first',                  # identificación única
        'NUM_PERIODOS': 'last',                     # último valor de número de períodos
        'MATERIAS': 'sum',                          # suma total de materias cursadas
        'CREDITOS_MATRICULADOS': 'sum',             # total de créditos matriculados
        'CREDITOS_APROBADOS_ACUM': 'last',          # último valor créditos aprobados acumulados
        'APROBADAS': 'sum',                         # total aprobadas (si es distinta de MATERIAS_APROBADAS)
        'PERDIDAS': 'sum',                          # total materias perdidas
        'R1': 'sum',                                # total materias en R1
        'R2': 'sum',                                # total materias en R2
        'R3': 'sum',                                # total materias en R3
        'R3_BALANCEO': 'sum',                       # total R3_balanceo
        'EFICIENCIA_ACADEMICA': 'mean',             # promedio de eficiencia académica
        'PROMEDIO_CARRERA': 'last'                  # último promedio de carrera
    }).reset_index()

    # --- Renombrar las columnas según tu esquema ---
    resumen = resumen.rename(columns={
        'APELLIDOS_NOMBRES': 'NOMBRES_APELLIDOS',
        'IDENTIFICACION': 'IDENTIFICACION',
        'NUM_PERIODOS': 'PERIODOS',
        'MATERIAS': 'NUM_MATERIAS',
        'CREDITOS_MATRICULADOS': 'NUM_CREDITOS_MATRICULADOS',
        'APROBADAS': 'NUM_MATERIAS_APROBADAS',
        'CREDITOS_APROBADOS_ACUM': 'NUM_CREDITOS_APROBADOS',        
        'PERDIDAS': 'NUM_MATERIAS_PERDIDAS',
        'R1': 'NUM_R1',
        'R2': 'NUM_R2',
        'R3': 'NUM_R3',
        'R3_BALANCEO': 'NUM_R3_BALANCEO',
        'EFICIENCIA_ACADEMICA': 'EFICIENCIA',
        'PROMEDIO_CARRERA': 'PROMEDIO'
    })

    # --- Ordenar alfabéticamente por nombres ---
    resumen = resumen.sort_values('NOMBRES_APELLIDOS', kind='stable').reset_index(drop=True)

    return resumen


In [16]:
df_fin_prueba = crear_base_general(df_pruebas)
df_fin_prueba.to_csv('/home/alexander/Escritorio/desercion-universitaria/data/features/df_fin_pruebas.csv')